In [1]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('keras_model.h5')

# Function to preprocess the input image
def preprocess_image(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    return image

# Function to predict using the model
def predict_image(image_path):
    # Preprocess the input image
    processed_image = preprocess_image(image_path)
    
    # Perform prediction
    prediction = model.predict(processed_image)
    
    # Decode the prediction (example: convert numerical class to label)
    # You need to modify this based on your model's output format
    predicted_class = np.argmax(prediction)
    
    # Return the predicted class or label
    return predicted_class

# Path to the image you want to predict
image_path = '0.jpg'

# Perform prediction
predicted_class = predict_image(image_path)
labels = ['a', 'aa', 'i','ii', 'u', 'uu', 'e', 'ee', 'ai', 'o', 'oo', 'au', 'ak']

# Print the predicted class or label
print("Predicted class:", labels[predicted_class])





1/1 [==============================] - 1s 1s/step
Predicted class: o


In [2]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('keras_model.h5')

# Function to predict using the model
def predict_frame(frame):
    # Perform prediction
    prediction = model.predict(frame)
    
    # Decode the prediction (example: convert numerical class to label)
    # You need to modify this based on your model's output format
    predicted_class = np.argmax(prediction)
    
    return predicted_class

# Main function to capture video from camera and perform prediction
def main():
    # Open camera capture
    minValue = 70
    capture = cv2.VideoCapture(0)
    
    while True:
        # Capture frame-by-frame
        _, frame = capture.read()

        # Simulating mirror Image
        frame = cv2.flip(frame, 1)
        
        # Coordinates of the ROI
        x1 = int(0.5 * frame.shape[1])
        y1 = 10
        x2 = frame.shape[1] - 10
        y2 = int(0.5 * frame.shape[1])

        # Drawing the ROI
        cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)
        
        # Extracting the ROI
        roi = frame[y1:y2, x1:x2]

        # Image Processing
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 2)
        th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        ret, test_image = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # Resize input image to (224, 224) dimensions
        test_image_resized = cv2.resize(test_image, (224, 224))

        # Convert resized image to a 3-channel image
        test_image_3_channel = cv2.cvtColor(test_image_resized, cv2.COLOR_GRAY2BGR)

        # Normalize the image
        test_image_normalized = test_image_3_channel.astype(np.float32) / 255.0

        # Expand dimensions to match model's input shape
        test_image_expanded = np.expand_dims(test_image_normalized, axis=0)

        # Perform prediction on the preprocessed frame
        predicted_class = predict_frame(test_image_expanded)

        labels = ['a', 'aa', 'i', 'ii', 'u', 'uu', 'e', 'ee', 'ai', 'o', 'oo', 'au', 'ak']
        predicted_label = labels[predicted_class]

        # Display the original frame
        cv2.imshow('Original Frame', test_image)

        # Display the predicted label on the original frame
        cv2.putText(frame, predicted_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame with predicted label
        cv2.imshow('Frame with Predicted Label', frame)

        # Check for exit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture
    capture.release()
    cv2.destroyAllWindows()

# Run the main function
if __name__ == "__main__":
    main()


1/1 [==============================] - 1s 671ms/step


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

# Define the CNN architecture
model = Sequential()

# Convolutional layer
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))

# Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening layer
model.add(Flatten())

# Fully connected layers
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=31, activation='softmax'))  # 31 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Image Data Augmentation
data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Splitting the data into 80% training and 20% validation

# Load and augment data from the same directory
train_generator = data_generator.flow_from_directory(
    'dataSet/trainingData',  # Use the same directory for both training and validation
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')  # Specify subset as 'training' for the training data

validation_generator = data_generator.flow_from_directory(
    'dataSet/trainingData',  # Use the same directory for both training and validation
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')  # Specify subset as 'validation' for the validation data

# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

# Save the trained model
model.save('trained_model_same_dir.h5')






Found 2743 images belonging to 31 classes.
Found 684 images belonging to 31 classes.


C:\Users\nandh\AppData\Local\Temp\ipykernel_20624\3601422785.py:48: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


86/86 [==============================] - 104s 1s/step - loss: 13.5379 - accuracy: 0.1203 - val_loss: 2.7024 - val_accuracy: 0.2822
Epoch 2/10
86/86 [==============================] - 84s 972ms/step - loss: 1.7249 - accuracy: 0.5680 - val_loss: 0.9663 - val_accuracy: 0.7646
Epoch 3/10
86/86 [==============================] - 77s 892ms/step - loss: 0.4779 - accuracy: 0.8990 - val_loss: 0.3896 - val_accuracy: 0.8991
Epoch 4/10
86/86 [==============================] - 80s 925ms/step - loss: 0.1864 - accuracy: 0.9614 - val_loss: 0.2660 - val_accuracy: 0.9342
Epoch 5/10
86/86 [==============================] - 95s 1s/step - loss: 0.0817 - accuracy: 0.9825 - val_loss: 0.2149 - val_accuracy: 0.9474
Epoch 6/10
86/86 [==============================] - 92s 1s/step - loss: 0.0737 - accuracy: 0.9818 - val_loss: 0.1367 - val_accuracy: 0.9620
Epoch 7/10
32/86 [==========>...................] - ETA: 49s - loss: 0.0636 - accuracy: 0.9823

In [5]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('trained_model_same_dir.h5')

# Function to predict using the model
def predict_frame(frame):
    # Perform prediction
    prediction = model.predict(frame)
    
    # Decode the prediction (example: convert numerical class to label)
    # You need to modify this based on your model's output format
    predicted_class = np.argmax(prediction)
    
    return predicted_class

# Main function to capture video from camera and perform prediction
def main():
    # Open camera capture
    minValue = 70
    capture = cv2.VideoCapture(0)
    
    while True:
        # Capture frame-by-frame
        _, frame = capture.read()

        # Simulating mirror Image
        frame = cv2.flip(frame, 1)
        
        # Coordinates of the ROI
        x1 = int(0.5 * frame.shape[1])
        y1 = 10
        x2 = frame.shape[1] - 10
        y2 = int(0.5 * frame.shape[1])

        # Drawing the ROI
        cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)
        
        # Extracting the ROI
        roi = frame[y1:y2, x1:x2]

        # Image Processing
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 2)
        th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        ret, test_image = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # Resize input image to (224, 224) dimensions
        test_image_resized = cv2.resize(test_image, (224, 224))

        # Convert resized image to a 3-channel image
        test_image_3_channel = cv2.cvtColor(test_image_resized, cv2.COLOR_GRAY2BGR)

        # Normalize the image
        test_image_normalized = test_image_3_channel.astype(np.float32) / 255.0

        # Expand dimensions to match model's input shape
        test_image_expanded = np.expand_dims(test_image_normalized, axis=0)

        # Perform prediction on the preprocessed frame
        predicted_class = predict_frame(test_image_expanded)

        labels = ['a', 'aa', 'i', 'ii', 'u', 'uu', 'e', 'ee', 'ai', 'o', 'oo', 'au', 'ak', 'ik', 'ing', 'ich', 'inj', 'it', 'inn', 'ith', 'ind', 'ip', 'im', 'iy', 'ir', 'il', 'iv', 'izh', 'ill', 'irr', 'in']
        predicted_label = labels[predicted_class]

        # Display the original frame
        cv2.imshow('Original Frame', test_image)

        # Display the predicted label on the original frame
        cv2.putText(frame, predicted_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame with predicted label
        cv2.imshow('Frame with Predicted Label', frame)

        # Check for exit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture
    capture.release()
    cv2.destroyAllWindows()

# Run the main function
if __name__ == "__main__":
    main()


In [3]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('trained_model_same_dir.h5')

# Function to predict using the model
def predict_frame(frame):
    # Perform prediction
    prediction = model.predict(frame)
    
    # Decode the prediction (example: convert numerical class to label)
    # You need to modify this based on your model's output format
    predicted_class = np.argmax(prediction)
    
    return predicted_class

# Main function to capture video from camera and perform prediction
def main():
    # Open camera capture
    minValue = 70
    capture = cv2.VideoCapture(0)
    
    while True:
        # Capture frame-by-frame
        _, frame = capture.read()

        # Simulating mirror Image
        frame = cv2.flip(frame, 1)
        
        # Coordinates of the ROI
        x1 = int(0.5 * frame.shape[1])
        y1 = 10
        x2 = frame.shape[1] - 10
        y2 = int(0.5 * frame.shape[1])

        # Drawing the ROI
        cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255, 0, 0), 1)
        
        # Extracting the ROI
        roi = frame[y1:y2, x1:x2]

        # Image Processing
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 2)
        th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        ret, test_image = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # Resize input image to (224, 224) dimensions
        test_image_resized = cv2.resize(test_image, (224, 224))

        # Convert resized image to a 3-channel image
        test_image_3_channel = cv2.cvtColor(test_image_resized, cv2.COLOR_GRAY2BGR)

        # Normalize the image
        test_image_normalized = test_image_3_channel.astype(np.float32) / 255.0

        # Expand dimensions to match model's input shape
        test_image_expanded = np.expand_dims(test_image_normalized, axis=0)

        # Perform prediction on the preprocessed frame
        predicted_class = predict_frame(test_image_expanded)

        labels = ['a', 'aa', 'i', 'ii', 'u', 'uu', 'e', 'ee', 'ai', 'o', 'oo', 'au', 'ak', 'ik', 'ing', 'ich', 'inj', 'it', 'inn', 'ith', 'ind', 'ip', 'im', 'iy', 'ir', 'il', 'iv', 'izh', 'ill', 'irr', 'in']
        predicted_label = labels[predicted_class]

        # Display the original frame
        cv2.imshow('Original Frame', test_image)

        # Display the predicted label on the original frame
        cv2.putText(frame, predicted_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the frame with predicted label
        cv2.imshow('Frame with Predicted Label', frame)

        # Check for exit key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture
    capture.release()
    cv2.destroyAllWindows()

# Run the main function
if __name__ == "__main__":
    main()


1/1 [==============================] - 1s 630ms/step


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Load the pre-trained model
model = load_model('trained_model.h5')

# Function to preprocess the input image
def preprocess_image(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    return image

# Function to predict using the model
def predict_image(image_path):
    # Preprocess the input image
    processed_image = preprocess_image(image_path)
    
    # Perform prediction
    prediction = model.predict(processed_image)
    
    # Decode the prediction (example: convert numerical class to label)
    # You need to modify this based on your model's output format
    predicted_class = np.argmax(prediction)
    
    # Return the predicted class or label
    return predicted_class

# Path to the image you want to predict
image_path = '0.jpg'

# Perform prediction
predicted_class = predict_image(image_path)
labels = ['a', 'aa', 'i','ii', 'u', 'uu', 'e', 'ee', 'ai', 'o', 'oo', 'au', 'ak']

# Print the predicted class or label
print("Predicted class:", labels[predicted_class])
